In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from utils import get_openai_api_key, get_serper_api_key

OPENAI_MODEL_NAME = 'gpt-4-turbo'

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = OPENAI_MODEL_NAME
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
# Document load
from populate_db import load_documents, split_documents

docs = load_documents()
print(f"Length of docs : {len(docs)}")
#print(docs[0])

# Split documents
chunks = split_documents(docs)
print(f"Length of chunks : {len(chunks)}")
#print(chunks[0])

In [ ]:
#Update CHROMA DB if need to be updated
from populate_db import add_to_chroma_db, clear_chroma_db

#clear_chroma_db()
add_to_chroma_db(chunks)

In [ ]:
# RAG Test not using Agent, just Query
from custom_rag_tool import query_rag
from IPython.display import Markdown

query_response = query_rag("5차 회의에서는 무슨 주제로 무제한 토론을 진행했나요?")

Markdown(query_response.content)

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from custom_rag_tool import CustomChromaDBRagTool

web_search_tool = SerperDevTool()
rag_tool = CustomChromaDBRagTool()

In [ ]:
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve the relevaent information from the vectorstore to answer the question",
    backstory=(
        "You are an assistant for question-answering tasks."
        "Use the information present in the retrieved context to answer the question."
        "You have to provide a clear concise answer."
    ),
    verbose=True,
    allow_delegation=False,
)

retriever_task = Task(
    description=(
        "Use the rag_tool, extract information for the question {question}."
    ),
    expected_output=(
        "Set of relevant documents and passages that can be used to answer the question {question}."
        "Return a clear and consise text as response."
        "Please give all answers in Korean."
    ),
    agent=retriever_agent,
    tools=[rag_tool],
)

In [ ]:
grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
)

grader_task = Task(
    description=(
        "Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=(
        "Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
        "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
        "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
        "Please provide explanation about the answer."
        "Please give all answers in Korean."
        #"Do not provide any preamble or explanations except for 'yes' or 'no'."
    ),
    agent=grader_agent,
    context=[retriever_task],
)

In [ ]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
)

hallucination_task = Task(
    description=(
        "Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."
    ),
    expected_output=(
        "Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
        "Respond 'yes' if the answer is in useful and contains fact about the question asked."
        "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
        "Please provide explanation about the answer."
        "Please give all answers in Korean."
        #"Do not provide any preamble or explanations except for 'yes' or 'no'."
    ),
    agent=hallucination_grader,
    context=[grader_task],
)

In [ ]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant then generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
)

answer_task = Task( 
    description=(
        "Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
        "If the answer is 'yes' return a clear and concise answer with Korean."
        "If the answer is 'no' then perform a 'websearch' and return the response"
    ),
    expected_output=(
        "Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
        "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
        "Otherwise respond as 'Sorry! unable to find a valid response'."
        "Please give all answers in Korean."
    ),  
    context=[hallucination_task],
    agent=answer_grader,
    tools=[web_search_tool],
)

In [ ]:
rag_crew = Crew(
    agents=[retriever_agent, grader_agent, hallucination_grader, answer_grader],
    tasks=[retriever_task, grader_task, hallucination_task, answer_task],
    memory=True,
    verbose=True,
)

In [ ]:
inputs = {"question":"대한민국 22대 415회 국회본회의 5차 회의에서는 무슨 주제로 무제한 토론을 진행했나요?"}

In [ ]:
result = rag_crew.kickoff(inputs=inputs)